In [66]:
import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt

def get_dataset_from_testcase(run):
    return run[0:run.rfind('_')]


def get_dataset_size(dataset):
    if dataset == "osm" or dataset == "books":
        return 800_000_000
    else:
        return 200_000_000

runs = []
dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge/join_all'
for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        run['dataset_size'] = get_dataset_size(run['dataset'])
        runs.append(run)

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge/eps_study'
for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        run['dataset_size'] = get_dataset_size(run['dataset'])
        runs.append(run)
df = pd.json_normalize(runs)
print(df.columns)

Index(['command', 'dataset', 'dataset_size', 'result.checksum',
       'result.duration_ns', 'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.num_output_keys', 'result.outer_disk_fetch',
       'result.outer_disk_fetch_size', 'result.outer_total_bytes_fetched',
       'spec.algo', 'spec.algo_name', 'spec.check_checksum', 'spec.common_key',
       'spec.index.epsilon', 'spec.index.search', 'spec.index.type',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk', 'spec.index.leaf_size_in_pages'],
      dtype='object')


In [67]:
df["threads"] = df["spec.num_threads"]
df["duration_sec"] = df["result.duration_ns"] / (1000000000)
df["ratio"] = df["spec.common_key"]
df["tput"] = (df["dataset_size"] / df["ratio"]) / (df["result.duration_ns"] / (1000000000))
df["algo"] = df["spec.algo_name"]
display(df["algo"].unique())
df["inner_bytes_fetched"] = df["result.inner_total_bytes_fetched"]
df["epsilon"] = df["spec.index.epsilon"]


array(['lsj_sampledflatpgm256', 'inlj_flatpgm4096', 'lsj_btree4096',
       'inlj_sampledflatpgm4096', 'inlj_flatpgm1024', 'hash_join',
       'inlj_pgm4096', 'sort_join', 'inlj_pgm1024', 'inlj_btree1024',
       'lsj_sampledflatpgm1024', 'inlj_btree256',
       'inlj_sampledflatpgm1024', 'lsj_pgm256', 'lsj_btree1024',
       'lsj_btree256', 'inlj_flatpgm256', 'lsj_flatpgm1024',
       'inlj_sampledflatpgm256', 'lsj_sampledflatpgm4096', 'inlj_pgm256',
       'lsj_pgm4096', 'lsj_pgm1024', 'lsj_flatpgm4096', 'lsj_flatpgm256',
       'inlj_btree4096', 'flatpgm256', 'flatpgm2048', 'flatpgm4096',
       'flatpgm1024', 'sj', 'flatpgm8192'], dtype=object)

In [68]:
rows = duckdb.sql("SELECT ratio, dataset, algo, avg(tput) as v FROM df WHERE algo='lsj_flatpgm256' OR algo='lsj_flatpgm2048' GROUP BY ratio, dataset,algo ORDER BY ratio").df()
rows.pivot(index='ratio', values='v', columns=['dataset', 'algo'])


dataset,normal,wiki,uniform_sparse,fb,osm,uniform_dense,books,lognormal
algo,lsj_flatpgm256,lsj_flatpgm256,lsj_flatpgm256,lsj_flatpgm256,lsj_flatpgm256,lsj_flatpgm256,lsj_flatpgm256,lsj_flatpgm256
ratio,,,,,,,,
1,2.399758e+07,4.989806e+07,2.392211e+07,2.391989e+07,1.087182e+07,2.365700e+07,7.640513e+06,2.552373e+07
10,1.707562e+07,1.876881e+07,1.570817e+07,1.636946e+07,8.068200e+06,1.593545e+07,5.085657e+06,1.633166e+07
100,6.263522e+06,7.972858e+06,6.138370e+06,6.037815e+06,3.019715e+06,6.572164e+06,2.241222e+06,6.413230e+06
1000,1.604043e+06,1.833758e+06,1.579772e+06,1.488961e+06,8.037759e+05,1.702927e+06,7.127977e+05,1.584487e+06


In [69]:
# Ratio = 1, epsilon vs duration across multiple threads
def plot_for_ratio(dataset):
    rows = duckdb.sql(
        " SELECT dataset, threads, ratio, epsilon, algo, AVG(duration_sec) as v FROM df"
        " WHERE (algo='lsj_flatpgm256' OR algo='lsj_flatpgm1024' OR algo='lsj_flatpgm4096')"
        f" AND dataset='{dataset}'"
        " GROUP BY dataset, ratio, threads, epsilon, algo"
        " ORDER BY ratio, threads, epsilon"
    ).df()
    return rows


In [71]:
# Ratio = 1, epsilon vs duration across multiple threads
display(plot_for_ratio('fb'))
display(plot_for_ratio('wiki'))

,dataset,threads,ratio,epsilon,algo,v
0,fb,1,1,256.0,lsj_flatpgm256,26.190664
1,fb,1,1,1024.0,lsj_flatpgm1024,28.362248
2,fb,1,1,4096.0,lsj_flatpgm4096,30.211240
3,fb,2,1,256.0,lsj_flatpgm256,13.609405
4,fb,2,1,1024.0,lsj_flatpgm1024,14.452971
5,fb,2,1,4096.0,lsj_flatpgm4096,17.613453
6,fb,4,1,256.0,lsj_flatpgm256,8.389189
7,fb,4,1,1024.0,lsj_flatpgm1024,8.784014
8,fb,4,1,4096.0,lsj_flatpgm4096,9.723148
9,fb,8,1,256.0,lsj_flatpgm256,5.996413
